# How to get started

For the installation of Julia or GigaSOM.jl please refer to the installation instructions.

## Cytometry Data

In this example we will use a subset of the Cytometry data from Bodenmiller et al.
(Bodenmiller et al., 2012). This data-set contains samples from peripheral blood
mononuclear cells (PBMCs) in unstimulated and stimulated conditions for 8 healthy donors.

10 cell surface markers (lineage markers) are used to identify different cell populations:
    - PBMC8_panel.xlsx (with Antigen name and columns for lineage markers and functional markers)
    - PBMC8_metadata.xlsx (file names, sample id, condition and patient id)

Before running this minimum working example, make sure to use the package:

In [1]:
using GigaSOM

## Input and output

The example data can be downloaded from [imlspenticton.uzh.ch/robinson_lab/cytofWorkflow/](http://imlspenticton.uzh.ch/robinson_lab/cytofWorkflow/)

You can fetch the files directly from within Julia:

In [2]:
# fetch the required data for testing and download the zip archive and unzip it
dataFiles = ["PBMC8_metadata.xlsx", "PBMC8_panel.xlsx", "PBMC8_fcs_files.zip"]
for f in dataFiles
    if !isfile(f)
        download("http://imlspenticton.uzh.ch/robinson_lab/cytofWorkflow/"*f, f)
        if occursin(".zip", f)
            run(`unzip PBMC8_fcs_files.zip`)
        end
    end
end

Read meta-data and panel as a `DataFrame`, and make sure that the column names match the CyTOF
FCS file names:

In [3]:
# Read  files as DataFrames
md = GigaSOM.DataFrame(GigaSOM.XLSX.readtable("PBMC8_metadata.xlsx", "Sheet1")...)
panel = GigaSOM.DataFrame(GigaSOM.XLSX.readtable("PBMC8_panel.xlsx", "Sheet1")...)
panel[:Isotope] = map(string, panel[:Isotope])
panel[:Metal] = map(string, panel[:Metal])
panel[:Antigen] = map(string, panel[:Antigen])
panel.Metal[1]=""
GigaSOM.insertcols!(panel,4,:fcs_colname => map((x,y,z)->x.*"(".*y.*z.*")".*"Dd",panel[:Antigen],panel[:Metal],panel[:Isotope]))
print(panel.fcs_colname)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[3]:4
└ @ Core In[3]:4
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[3]:4
└ @ Core In[3]:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[3]:5
└ @ Core In[3]:5
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[3]:5
└ @ Core In[3]:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[3]:6
└ @ Core In[3]:6
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated,

["CD3(110:114)Dd", "CD45(In115)Dd", "BC1(La139)Dd", "BC2(Pr141)Dd", "pNFkB(Nd142)Dd", "pp38(Nd144)Dd", "CD4(Nd145)Dd", "BC3(Nd146)Dd", "CD20(Sm147)Dd", "CD33(Nd148)Dd", "pStat5(Nd150)Dd", "CD123(Eu151)Dd", "pAkt(Sm152)Dd", "pStat1(Eu153)Dd", "pSHP2(Sm154)Dd", "pZap70(Gd156)Dd", "pStat3(Gd158)Dd", "BC4(Tb159)Dd", "CD14(Gd160)Dd", "pSlp76(Dy164)Dd", "BC5(Ho165)Dd", "pBtk(Er166)Dd", "pPlcg2(Er167)Dd", "pErk(Er168)Dd", "BC6(Tm169)Dd", "pLat(Er170)Dd", "IgM(Yb171)Dd", "pS6(Yb172)Dd", "HLA-DR(Yb174)Dd", "BC7(Lu175)Dd", "CD7(Yb176)Dd", "DNA-1(Ir191)Dd", "DNA-2(Ir193)Dd"]

Extract the lineage and functional markers with `getMarkers()` function:

In [4]:
lineageMarkers, functionalMarkers = getMarkers(panel)

(["CD3(110:114)Dd", "CD45(In115)Dd", "CD4(Nd145)Dd", "CD20(Sm147)Dd", "CD33(Nd148)Dd", "CD123(Eu151)Dd", "CD14(Gd160)Dd", "IgM(Yb171)Dd", "HLA_DR(Yb174)Dd", "CD7(Yb176)Dd"], ["pNFkB(Nd142)Dd", "pp38(Nd144)Dd", "pStat5(Nd150)Dd", "pAkt(Sm152)Dd", "pStat1(Eu153)Dd", "pSHP2(Sm154)Dd", "pZap70(Gd156)Dd", "pStat3(Gd158)Dd", "pSlp76(Dy164)Dd", "pBtk(Er166)Dd", "pPlcg2(Er167)Dd", "pErk(Er168)Dd", "pLat(Er170)Dd", "pS6(Yb172)Dd"])

Read FCS files `readFlowset()`:

In [5]:
fcsRaw = readFlowset(md.file_name)

Dict{Any,Any} with 16 entries:
  "PBMC8_30min_patient8_Reference.fcs" => 13670×35 DataFrames.DataFrame. Omitte…
  "PBMC8_30min_patient2_BCR-XL.fcs"    => 16675×35 DataFrames.DataFrame. Omitte…
  "PBMC8_30min_patient5_BCR-XL.fcs"    => 8543×35 DataFrames.DataFrame. Omitted…
  "PBMC8_30min_patient1_Reference.fcs" => 2739×35 DataFrames.DataFrame. Omitted…
  "PBMC8_30min_patient6_BCR-XL.fcs"    => 8622×35 DataFrames.DataFrame. Omitted…
  "PBMC8_30min_patient4_Reference.fcs" => 6906×35 DataFrames.DataFrame. Omitted…
  "PBMC8_30min_patient3_BCR-XL.fcs"    => 12252×35 DataFrames.DataFrame. Omitte…
  "PBMC8_30min_patient7_Reference.fcs" => 15974×35 DataFrames.DataFrame. Omitte…
  "PBMC8_30min_patient1_BCR-XL.fcs"    => 2838×35 DataFrames.DataFrame. Omitted…
  "PBMC8_30min_patient5_Reference.fcs" => 11962×35 DataFrames.DataFrame. Omitte…
  "PBMC8_30min_patient6_Reference.fcs" => 11038×35 DataFrames.DataFrame. Omitte…
  "PBMC8_30min_patient7_BCR-XL.fcs"    => 14770×35 DataFrames.DataFrame. Omitt

`readFlowset()` is a wrapper function around [FCSFiles.jl](https://github.com/tlnagy/FCSFiles.jl). Please note the current limitations
of this package (i.e., the [limit for large files](https://github.com/tlnagy/FCSFiles.jl/blob/master/src/parse.jl#L20)).

Clean names to remove problematic characters in the column names:

In [6]:
cleanNames!(fcsRaw)

Finally, create a `daFrame` that contains the expression data as well as panel
and meta-data. It automatically applies a `asinh` tranformation with a cofactor of 5.

In [7]:
daf = createDaFrame(fcsRaw, md, panel)

┌ Warning: Implicit broadcasting to a new column in DataFrame assignment is deprecated.Use an explicit broadcast with `df[!, col_ind] .= v`
│   caller = createDaFrame(::Dict{Any,Any}, ::DataFrames.DataFrame, ::DataFrames.DataFrame) at process.jl:88
└ @ GigaSOM /Users/laurent.heirendt/.julia/packages/GigaSOM/QAKEY/src/io/process.jl:88


daFrame(172791×25 DataFrames.DataFrame. Omitted printing of 21 columns
│ Row    │ CD3(110:114)Dd │ CD45(In115)Dd │ CD4(Nd145)Dd │ CD20(Sm147)Dd │
│        │ Float32        │ Float32       │ Float32      │ Float32       │
├────────┼────────────────┼───────────────┼──────────────┼───────────────┤
│ 1      │ 0.863966       │ 4.59768       │ -0.157656    │ -0.131486     │
│ 2      │ 1.90267        │ 5.88631       │ 2.13232      │ 2.4149        │
│ 3      │ 4.96538        │ 6.63111       │ -0.100279    │ 0.993387      │
│ 4      │ 2.92577        │ 5.08396       │ -0.0759843   │ 1.50545       │
│ 5      │ 4.19087        │ 6.53202       │ 2.49969      │ 2.24803       │
│ 6      │ 3.78095        │ 5.96461       │ 1.66088      │ 0.201739      │
│ 7      │ -1.04096       │ 5.53396       │ 1.65052      │ 5.1049        │
│ 8      │ 4.36623        │ 6.24286       │ 4.87603      │ -0.0164116    │
│ 9      │ 1.36755        │ 1.2471        │ 3.8174       │ -0.112002     │
│ 10     │ 3.98743        │ 5

## Creating a Self Organizing MAP (SOM)

The main advantage of `GigaSOM.jl` is the capability of parallel processing.
In order to activate this dependency, please activate the GigaSOM environment:

In [8]:
import Pkg; Pkg.activate("GigaSOM")

┌ Info: activating new environment at ~/work/git/hub/GigaSOM.jl/docs/src/GigaSOM.
└ @ Pkg.API /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Pkg/src/API.jl:519


"/Users/laurent.heirendt/work/git/hub/GigaSOM.jl/docs/src/GigaSOM"

Alternatively, on the REPL, you can also activate the `GigaSOM` environment by typing `]`:
```julia
v(1.1) pkg> activate GigaSOM
```

Without the explicit declaration of multiple workers, `GigaSOM` will train the SOM grid on a single
core. Therefore, we will add some workers and make sure that `GigaSOM` is accessible to
all the workers:

In [9]:
using Distributed
addprocs(2) # the number of workers can be higher than 2
@everywhere using GigaSOM

We will use only the lineage markers (cell surface) for the training of the SOM map
and extract the expression data:

In [10]:
cc = map(Symbol, lineageMarkers)
dfSom = daf.fcstable[:,cc]

,CD3(110:114)Dd,CD45(In115)Dd,CD4(Nd145)Dd,CD20(Sm147)Dd,CD33(Nd148)Dd,CD123(Eu151)Dd
,Float32,Float32,Float32,Float32,Float32,Float32
1,0.863966,4.59768,-0.157656,-0.131486,1.496,0.0192691
2,1.90267,5.88631,2.13232,2.4149,0.718917,-0.174217
3,4.96538,6.63111,-0.100279,0.993387,0.995998,0.886214
4,2.92577,5.08396,-0.0759843,1.50545,-0.144179,0.211041
5,4.19087,6.53202,2.49969,2.24803,0.570482,-0.106751
6,3.78095,5.96461,1.66088,0.201739,0.0156762,0.28967
7,-1.04096,5.53396,1.65052,5.1049,-0.0939763,-0.863822
8,4.36623,6.24286,4.87603,-0.0164116,1.24873,0.414543
9,1.36755,1.2471,3.8174,-0.112002,-0.0941085,0.44179


Initialize the SOM grid by size and expression values:

In [11]:
som2 = initGigaSOM(dfSom, 10, 10)

GigaSOM.Som([0.772605 5.55571 … 0.812848 -0.288866; 1.42088 4.97668 … 1.07725 4.22555; … ; -0.738192 3.88047 … 0.795165 -0.145001; 0.535444 5.25279 … 0.430748 2.74619], ["CD3(110:114)Dd", "CD45(In115)Dd", "CD4(Nd145)Dd", "CD20(Sm147)Dd", "CD33(Nd148)Dd", "CD123(Eu151)Dd", "CD14(Gd160)Dd", "IgM(Yb171)Dd", "HLA_DR(Yb174)Dd", "CD7(Yb176)Dd"], 2×10 DataFrames.DataFrame. Omitted printing of 6 columns
│ Row │ CD3(110:114)Dd │ CD45(In115)Dd │ CD4(Nd145)Dd │ CD20(Sm147)Dd │
│     │ Float64        │ Float64       │ Float64      │ Float64       │
├─────┼────────────────┼───────────────┼──────────────┼───────────────┤
│ 1   │ 0.0            │ 0.0           │ 0.0          │ 0.0           │
│ 2   │ 1.0            │ 1.0           │ 1.0          │ 1.0           │, :none, 10, 10, 100, [0.0 0.0; 1.0 0.0; … ; 8.0 9.0; 9.0 9.0], 100×2 DataFrames.DataFrame
│ Row │ X     │ Y     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 1     │ 1     │
│ 2   │ 2     │ 2     │
│ 3   │ 3     │ 3     │
│ 4   │

Train the SOM grid with the initialized SOM object and define the number of training
rounds (also referred to as *epochs*).

In [12]:
 som2 = trainGigaSOM(som2, dfSom, epochs = 10)

┌ Info: The radius has been determined automatically.
└ @ GigaSOM /Users/laurent.heirendt/.julia/packages/GigaSOM/QAKEY/src/core.jl:79


Epoch: 1
Radius: 6.463961030678928
Epoch: 2
Radius: 5.856854249492381
Epoch: 3
Radius: 5.249747468305833
Epoch: 4
Radius: 4.642640687119286
Epoch: 5
Radius: 4.035533905932739
Epoch: 6
Radius: 3.4284271247461913
Epoch: 7
Radius: 2.821320343559644
Epoch: 8
Radius: 2.2142135623730965
Epoch: 9
Radius: 1.607106781186549
Epoch: 10
Radius: 1.0000000000000013


GigaSOM.Som([0.648143 5.0357 … 2.59634 0.466907; 0.928678 5.05525 … 2.38048 0.78602; … ; 0.738282 4.70267 … 0.671534 0.777843; 0.92694 4.86462 … 0.568667 0.467445], ["CD3(110:114)Dd", "CD45(In115)Dd", "CD4(Nd145)Dd", "CD20(Sm147)Dd", "CD33(Nd148)Dd", "CD123(Eu151)Dd", "CD14(Gd160)Dd", "IgM(Yb171)Dd", "HLA_DR(Yb174)Dd", "CD7(Yb176)Dd"], 2×10 DataFrames.DataFrame. Omitted printing of 6 columns
│ Row │ CD3(110:114)Dd │ CD45(In115)Dd │ CD4(Nd145)Dd │ CD20(Sm147)Dd │
│     │ Float64        │ Float64       │ Float64      │ Float64       │
├─────┼────────────────┼───────────────┼──────────────┼───────────────┤
│ 1   │ 0.0            │ 0.0           │ 0.0          │ 0.0           │
│ 2   │ 1.0            │ 1.0           │ 1.0          │ 1.0           │, :none, 10, 10, 100, [0.0 0.0; 1.0 0.0; … ; 8.0 9.0; 9.0 9.0], 100×2 DataFrames.DataFrame
│ Row │ X     │ Y     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 1     │ 1     │
│ 2   │ 2     │ 2     │
│ 3   │ 3     │ 3     │
│ 4   │ 4  

Finally, calculate the winner neurons from the trained SOM object:

In [13]:
winners = mapToGigaSOM(som2, dfSom)

[2, 3]


,index
,Int64
1,100
2,91
3,61
4,36
5,18
6,16
7,1
8,9
9,94
